This document demonstrates the making, training, saving, loading, and usage of a sklearn-compliant CGCNN model.

In [1]:
import os
import sys
# sys.path.insert(0,'/home/junwoony/.local/lib/python3.6/site-packages')
sys.path.insert(0,'../')
import numpy as np
#Select which GPU to use if necessary
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1
%env CUDA_LAUNCH_BLOCKING=1
import mongo
# from torchviz import make_dot, make_dot_from_trace
import time
from torch.nn.utils import clip_grad_norm_
import pickle
import random
from torch.utils.data import Dataset, DataLoader
import mongo
# from cgcnn.data_icgcnn import StructureData, ListDataset, StructureDataTransformer
from cgcnn.data_grad_surface import StructureData, ListDataset, StructureDataTransformer
import numpy as np
import tqdm
from sklearn.preprocessing import StandardScaler
from pymatgen.io.ase import AseAtomsAdaptor

import multiprocess as mp
from sklearn.model_selection import ShuffleSplit

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1
env: CUDA_LAUNCH_BLOCKING=1


## Load the dataset as mongo docs

In [4]:
docs = pickle.load(open('../input/intermetallics_cleavage_energy_data.pkl', 'rb'))

# random.seed(123)
# random.shuffle(docs)
for doc in docs:
    doc["atoms"] = doc['thinnest_structure']['atoms']
    doc["results"] = doc['thinnest_structure']['results']
    doc["initial_configuration"] = doc['thinnest_structure']['initial_configuration']
    del doc["thinnest_structure"]
    
SDT_list = pickle.load(open('../input/SDT_surface_new.pkl', 'rb'))
structures = SDT_list[0]
orig_atom_fea_len = structures[0].shape[-1]
nbr_fea_len = structures[1].shape[-1]
target_list = np.array([sdt[-1].numpy() for sdt in SDT_list]).reshape(-1,1)

In [3]:
print(len(docs), len(SDT_list), len(target_list))

3033 3033 3033


## Get the size of the features from the data transformer, to be used in setting up the net model

determine max connectivity value (for radius)

In [3]:
# from torch.utils.data import Dataset, DataLoader
# import mongo
# from cgcnn.data import StructureData, ListDataset, StructureDataTransformer
# import numpy as np
# import tqdm



# SDT = StructureDataTransformer(atom_init_loc='/home/zulissi/software/cgcnn_sklearn/atom_init.json',
#                               max_num_nbr=12,
#                               step=0.2,
#                               radius=1,
#                               use_tag=False,
#                               use_fixed_info=False)

# SDT_out = SDT.transform(docs)

# structures = SDT_out[0]
# orig_atom_fea_len = structures[0].shape[-1]
# nbr_fea_len = structures[1].shape[-1]



In [4]:
print(orig_atom_fea_len, nbr_fea_len)

93 19


## CGCNN model with skorch to make it sklearn compliant

In [5]:
from torch.optim import Adam, SGD
from sklearn.model_selection import ShuffleSplit
from skorch.callbacks import Checkpoint, LoadInitState #needs skorch 0.4.0, conda-forge version at 0.3.0 doesn't cut it

from cgcnn.data_grad_surface import collate_pool, MergeDataset
from cgcnn.model_grad_2_surface_simple_sigopt import CrystalGraphConvNet
from skorch import NeuralNetRegressor
import torch
import skorch.callbacks.base


cuda = torch.cuda.is_available()
if cuda:
    device = torch.device("cuda")
else:
    device='cpu'

#Make a checkpoint to save parameters every time there is a new best for validation lost
cp = Checkpoint(monitor='valid_loss_best',fn_prefix='valid_best_')

#Callback to load the checkpoint with the best validation loss at the end of training

class train_end_load_best_valid_loss(skorch.callbacks.base.Callback):
    def on_train_end(self, net, X, y):
        net.load_params('valid_best_params.pt')
        
load_best_valid_loss = train_end_load_best_valid_loss()
print('device', device)

device cuda


## Example converting all the documents up front

In [6]:
# #Make the target list
# import seaborn as sns
# from sklearn.preprocessing import StandardScaler, MinMaxScaler

# target_list = np.array(energies).reshape(-1,1)
# sns.distplot(target_list, color='black')
    
# #scaler = StandardScaler().fit(energies.reshape(-1, 1))
# scaler = MinMaxScaler().fit(energies.reshape(-1, 1))
# target_list = scaler.transform(energies.reshape(-1,1))
# print(type(target_list))

# sns.distplot(target_list, color='red')

In [7]:
# inversed_target_list = scaler.inverse_transform(target_list)
# sns.distplot(inversed_target_list)


In [8]:
from sklearn.model_selection import train_test_split

SDT_list= SDT_list
target_list = target_list

indices = np.arange(len(SDT_list))
SDT_training, SDT_test, target_training, target_test, train_idx, test_idx \
= train_test_split(SDT_list, target_list, indices, test_size=0.2, random_state=42)

In [9]:
from skorch.dataset import CVSplit
from skorch.callbacks.lr_scheduler import WarmRestartLR, LRScheduler
from adamwr.adamw import AdamW
from torch.optim.lbfgs import LBFGS

from adamwr.cosine_scheduler import CosineLRWithRestarts
from sklearn.model_selection import train_test_split

train_test_splitter = ShuffleSplit(test_size=0.25, random_state=42)

# batchsize = (10,300)
# # warm restart scheduling from https://arxiv.org/pdf/1711.05101.pdf
# LR_schedule = LRScheduler(CosineLRWithRestarts, batch_size=batchsize, epoch_size=len(SDT_training), restart_period=10, t_mult=1.2)
LR_schedule = LRScheduler('MultiStepLR',milestones=[100],gamma=0.1)

#############
# To extract intermediate features, set the forward takes only the first return value to calculate loss
class MyNet(NeuralNetRegressor):
    def get_loss(self, y_pred, y_true, **kwargs):        
        y_pred = y_pred[0] if isinstance(y_pred, tuple) else y_pred  # discard the 2nd output
        differ=torch.sum((y_pred-y_true.cuda())**2.0,dim=1)
        if torch.nonzero(differ).shape[0] != differ.shape[0]:
            print('zero sqrt for Loss')
#             zero_idx = (differ == 0).nonzero()
#             differ[zero_idx] = 1e-6
        differ = torch.clamp(differ, min=1e-8)
        return torch.mean(torch.sqrt(differ))
#         return torch.mean(torch.sqrt(torch.sum((y_pred-y_true.cuda())**2.0,dim=1)))
#         return super().get_loss(y_pred, y_true, **kwargs)
## return features = net.forward(SDT_test)


net = MyNet(
    CrystalGraphConvNet,
    module__orig_atom_fea_len = orig_atom_fea_len,
    module__nbr_fea_len = nbr_fea_len,
#     module__angle_fea_len = 8, #angle_fea_len,
    batch_size=(10,300), #214
    module__classification=False,
    lr=(np.exp(-15),np.exp(-3)),
    max_epochs= (50, 800),
    module__atom_fea_len=(4,256), #46,
    module__h_fea_len=(32, 256),
    module__n_conv=(1,10), #8
    module__n_h=(1,10),
    module__max_num_nbr=12, #9
    module__opt_step_size=(0.01, 0.9), #0.3
    module__min_opt_steps=30,
    module__max_opt_steps=300,
    module__momentum=(0.1,0.9),
    module__dropout=(0, 0.3),
    module__dropout_h=(0, 0.3),
    optimizer__weight_decay=(1e-6, 1e-2),
    optimizer=AdamW,
    iterator_train__pin_memory=True,
    iterator_train__num_workers=0,
    iterator_train__shuffle=True,
    iterator_train__collate_fn = collate_pool,
    iterator_valid__pin_memory=True,
    iterator_valid__num_workers=0,
    iterator_valid__collate_fn = collate_pool,
    device=device,
#     criterion=torch.nn.MSELoss,
    criterion=torch.nn.L1Loss,
    dataset=MergeDataset,
    train_split = CVSplit(cv=train_test_splitter),
    callbacks=[cp, LR_schedule, load_best_valid_loss] #    callbacks=[cp, load_best_valid_loss, LR_schedule]

)

## Best parameters

# Shuffle and Split

In [10]:
from sigopt_sklearn.search import SigOptSearchCV
from sklearn.metrics import get_scorer
from sklearn.model_selection import ShuffleSplit

client_token = "TSRIPFKLRAIMUDDVQEBJHVBQRVBCDJOSKJMKEQTXWCYZDNED"
net_parameters = {
                'batch_size':(10,300),
                'lr':(np.exp(-15),np.exp(-3)),
                'max_epochs': (50, 800),
                'module__atom_fea_len':(4,256), #46,
                'module__h_fea_len':(32, 256),
                'module__n_conv':(1,10), #8
                'module__n_h':(1,10),
                'module__opt_step_size':(0.01, 0.9), #0.3
                'module__momentum':(0, 0.9),
                'module__dropout':(0, 0.3),
                'module__dropout_h':(0, 0.3),
                'optimizer__weight_decay':(1e-6, 1e-2)
                }

clf = SigOptSearchCV(net, net_parameters, cv=train_test_splitter, client_token=client_token,
                    n_jobs=1, n_iter=50, scoring=get_scorer('neg_mean_absolute_error'))

len(net_parameters)

12

In [11]:
clf.fit(SDT_training, target_training)

/home/junwoony/miniconda3/envs/schnet2/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:82: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule.See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


  epoch    train_loss    valid_loss    cp     dur
-------  ------------  ------------  ----  ------
      1        0.6802        1.2459     +  5.0623
      2        0.3004        0.7220     +  15.4242
      3        0.5135        0.1931     +  15.6274
      4        0.1544        0.1553     +  15.8103
      5        0.1467        0.1508     +  15.9345
      6        0.1494        0.1517        15.7745
      7        0.1501        0.1521        15.6667
      8        0.1478        0.1523        15.6091
      9        0.1505        0.1523        15.6403
     10        0.1565        0.1523        15.6972
     11        0.1515        0.1523        15.7942
     12        0.1501        0.1522        15.6900
     13        0.1496        0.1521        15.8961
     14        0.1489        0.1520        15.7871
     15        0.1497        0.1518        15.6850
     16        0.1484        0.1515        15.6130
     17        0.1482        0.1512        15.7637
     18        0.1517        0.150

    147        0.1194        0.1318        15.7032
    148        0.1191        0.1319        15.7323
    149        0.1177        0.1327        15.7728
    150        0.1187        0.1327        15.7308
    151        0.1186        0.1327        15.7951
    152        0.1194        0.1327        15.9803
    153        0.1176        0.1328        15.7441
    154        0.1182        0.1332        15.7614
    155        0.1253        0.1328        15.7473
    156        0.1183        0.1317        15.6710
    157        0.1180        0.1324        15.8182
    158        0.1194        0.1322        15.8515
    159        0.1136        0.1328        15.7835
    160        0.1190        0.1323        15.8576
    161        0.1150        0.1325        15.8225
    162        0.1158        0.1326        15.7819
    163        0.1175        0.1324        15.6516
    164        0.1161        0.1315        15.6828
    165        0.1172        0.1323        15.8302
    166        0.1169        0.

ValueError: Found input variables with inconsistent numbers of samples: [607, 9211]

In [14]:
9211/607

15.174629324546952

In [1]:
clf.best_params_, clf.best_score_


NameError: name 'clf' is not defined

In [ ]:
batch_size	74
lr	0.010390047081273095
max_epochs	272
module__atom_fea_len	82
module__h_fea_len	120
module__n_conv	4
module__n_h	3